In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from efficient_apriori import apriori
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from scipy.stats import f_oneway
import datetime


In [2]:
df = pd.read_csv('Project data_Raw.csv',skiprows=3)

/var/folders/c_/8r42hlvx2w36c1j39xf03hkw0000gp/T/ipykernel_20874/3052439813.py:1: DtypeWarning: Columns (19,21,22,23,24,25,26,27,31,35,37,39,43,44,45,51,52,55,59,68,77,84,85,89,92,94,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Project data_Raw.csv',skiprows=3)


In [3]:
df.head()

,MAINCATEGORY,SUBCATEGORY,BRANDS,CATEGORY,FAMILY,Bill Date,Outlet Name,Location Name,Order Type,Counter,...,Item Weight in GMs,Price Level Name,Rate Edit Reason,Max Conversion Type,Max Conversion Qty,Bill Qty with Max Conv,Min Conversion Qty,Bill Qty with Min Conv,Unique Trans Refno,Conversion Rate
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Processed Food,Jam / Spreads,NaN,Ready Foods,NaN,2025-03-02,FRESH2DAY-PALAVAKKAM,Main Location,NaN,BAKOF,...,1.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,,120.0
2,Processed Food,Jam / Spreads,NaN,Ready Foods,NaN,2025-03-02,FRESH2DAY-PALAVAKKAM,Main Location,NaN,BAKOF,...,1.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,,120.0
3,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,NaN,2025-03-14,FRESH2DAY - KILPAUK,Main Location,NaN,CL4,...,0.0,NaN,NaN,NaN,0.0,0.6,0.0,0.6,,80.0
4,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,NaN,2025-03-14,FRESH2DAY - KILPAUK,Main Location,NaN,CL4,...,0.0,NaN,NaN,NaN,0.0,0.09,0.0,0.09,,80.0


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = df.iloc[1:]
df.head()

,MAINCATEGORY,SUBCATEGORY,BRANDS,CATEGORY,FAMILY,Bill Date,Outlet Name,Location Name,Order Type,Counter,PBill No(T),Cust Name,Bill Time,Outlet ERP Code,User Name,Item Code,Item Name,HSN_SAC Code,AliasName,Qty,Free Qty,Bill Amt(WOT Charges),Item Amt,Item Rate,Item Disc Amt,Bill Disc Amt,SMS_ALERT_NO,Total Disc Amt,Tax%,Cust cid,Cust_Code,Tax Amt,Item_Master_Base_UOM,Service Tax%,Service Tax Amt,Amt With Tax,CESS %,CESS Amt,CGST %,CGST Amt,IGST %,IGST Amt,SGST %,SGST Amt,Total Amt,Gross Margin,Void Qty,Void Amt,I_U,Item Remarks,Sales Type,I_U Qty,GoodsValue,Combo Qty,Supplier,Total Qty,Item Conversion,RateDiff(Ori Grathan Billed),RateDiff(Ori Lesthan Billed),Piece_Rate_WOTtax,Batch No,Expiry Dt,Tender Type,Item Free Qty in(Weight)KG,EAN Code,Group Name,Type Name,Extra Cess Amount,Item Conversion Qty,Online_order_number,SO_Channel,GST BILL NO,Item GST UOM,Calamity Cess Amount,Customer GST No,Item MRP,Gross_Profit%,Purchase amount WOT,GST Tax Amt WOT Cess,GST Cess Amt,Location Type,Customer Address,Customer City,Freight Charge,Landing Cost,Purchase Rate WOT Tax,Item Qty in(Weight)KG,Item Weight in GMs,Price Level Name,Rate Edit Reason,Max Conversion Type,Max Conversion Qty,Bill Qty with Max Conv,Min Conversion Qty,Bill Qty with Min Conv,Unique Trans Refno,Conversion Rate
1,Processed Food,Jam / Spreads,NaN,Ready Foods,NaN,2025-03-02,FRESH2DAY-PALAVAKKAM,Main Location,NaN,BAKOF,CA6,FRESH2DAY,22:23,NaN,10344,38784.0,F2D LEMON PICKLE 300 GM,20019000,,1.00,0.0,99.00,120.00,120.0,0.00,0.00,9091600916.0,21.00,12.0,1N60006307,18268.0,10.60,,0.0,0.0,120.00,0.0,0.00,6.0,5.30,0.0,0.0,6.0,5.30,99.00,31.13,0.0,0.0,1.0,,Direct Sales,1.00,88.40,0.0,NaN,1.0,,0.0,0.0,88.4,F38784-7454,2035-02-28,Cash,0.0,,UNMAPPED,DEFAULT,0.0,1.0,NaN,NaN,ANW/CA6,UNT,0.0,,120.0,35.21,57.27,10.6,0.0,LOCAL,,,0.0,64.14,57.27,0.00,1.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,,120.0
2,Processed Food,Jam / Spreads,NaN,Ready Foods,NaN,2025-03-02,FRESH2DAY-PALAVAKKAM,Main Location,NaN,BAKOF,CA6,FRESH2DAY,22:23,NaN,10344,38793.0,F2D TOMATO PICKLE 300 GM,20019000,,1.00,0.0,99.00,120.00,120.0,0.00,0.00,9091600916.0,21.00,12.0,1N60006307,18268.0,10.60,,0.0,0.0,120.00,0.0,0.00,6.0,5.30,0.0,0.0,6.0,5.30,99.00,30.56,0.0,0.0,1.0,,Direct Sales,1.00,88.40,0.0,NaN,1.0,,0.0,0.0,88.4,F38793-7463,2035-02-28,Cash,0.0,,UNMAPPED,DEFAULT,0.0,1.0,NaN,NaN,ANW/CA6,UNT,0.0,,120.0,34.57,57.84,10.6,0.0,LOCAL,,,0.0,64.78,57.84,0.00,1.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,,120.0
3,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,NaN,2025-03-14,FRESH2DAY - KILPAUK,Main Location,NaN,CL4,CA15,FRESH2DAY,20:51,NaN,JANAGIRAMANAL,156.0,BANANA KARPOORAVALI,07051100,BAN KAR,0.60,0.0,48.24,48.24,80.0,0.00,0.00,9091600916.0,0.00,0.0,1N60006307,18452.0,0.00,Bag,0.0,0.0,48.24,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,48.24,16.13,0.0,0.0,1.0,,Direct Sales,0.60,48.24,0.0,NaN,0.6,,0.0,0.0,80.0,B156-8217,2035-03-31,Cash,0.0,,UNMAPPED,DEFAULT,0.0,0.6,NaN,NaN,FKI/CA15,UNT,0.0,,80.0,33.43,32.11,0.0,0.0,LOCAL,,,0.0,53.25,53.25,0.60,0.0,NaN,NaN,NaN,0.0,0.6,0.0,0.6,,80.0
4,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,NaN,2025-03-14,FRESH2DAY - KILPAUK,Main Location,NaN,CL4,CA15,FRESH2DAY,20:51,NaN,JANAGIRAMANAL,156.0,BANANA KARPOORAVALI,07051100,BAN KAR,0.09,0.0,6.96,6.96,80.0,0.00,0.00,9091600916.0,0.00,0.0,1N60006307,18452.0,0.00,Bag,0.0,0.0,6.96,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,6.96,2.33,0.0,0.0,1.0,,Direct Sales,0.09,6.96,0.0,NaN,0.09,,0.0,0.0,80.0,B156-8326,2035-03-31,Cash,0.0,,UNMAPPED,DEFAULT,0.0,0.09,NaN,NaN,FKI/CA15,UNT,0.0,,80.0,33.43,4.63,0.0,0.0,LOCAL,,,0.0,53.25,53.25,0.09,0.0,NaN,NaN,NaN,0.0,0.09,0.0,0.09,,80.0
5,FRUITS & VEGETABLES,Herbs & Seasoning,F2D,FRESH VEGETABLES,NaN,2025-03-14,FRESH2DAY - KILPAUK,Main Location,NaN,CL4,CA15,FRESH2DAY,20:51,NaN,JANAGIRAMANAL,88.0,GARLICK,07051100,GAR,0.66,0.0,112.22,112.22,169.0,0.00,0.00,9091600916.0,0.00,0.0,1N60006307,18452.0,0.00,Bag,0.0,0.0,112.22,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,112.22,22.81,0.0,0.0,1.0,,Direct Sales,0.66,112.22,0.0,NaN,0.66,,0.0,0.0,169.01,G88-7459,2035-03-3

In [5]:
df.columns

Index(['MAINCATEGORY', 'SUBCATEGORY', 'BRANDS', 'CATEGORY', 'FAMILY',
       'Bill Date', 'Outlet Name', 'Location Name', 'Order Type', 'Counter',
       'PBill No(T)', 'Cust Name', 'Bill Time', 'Outlet ERP Code', 'User Name',
       'Item Code', 'Item Name', 'HSN_SAC Code', 'AliasName', 'Qty',
       'Free Qty', 'Bill Amt(WOT Charges)', 'Item Amt', 'Item Rate',
       'Item Disc Amt', 'Bill Disc Amt', 'SMS_ALERT_NO', 'Total Disc Amt',
       'Tax%', 'Cust cid', 'Cust_Code', 'Tax Amt', 'Item_Master_Base_UOM',
       'Service Tax%', 'Service Tax Amt', 'Amt With Tax', 'CESS %', 'CESS Amt',
       'CGST %', 'CGST Amt', 'IGST %', 'IGST Amt', 'SGST %', 'SGST Amt',
       'Total Amt', 'Gross Margin', 'Void Qty', 'Void Amt', 'I_U',
       'Item Remarks', 'Sales Type', 'I_U Qty', 'GoodsValue', 'Combo Qty',
       'Supplier', 'Total Qty', 'Item Conversion',
       'RateDiff(Ori Grathan Billed)', 'RateDiff(Ori Lesthan Billed)',
       'Piece_Rate_WOTtax', 'Batch No', 'Expiry Dt', 'Tender Type',


# Data cleaning

In [6]:
df_new = df[['Bill Date','Cust cid','Outlet Name','MAINCATEGORY', 'SUBCATEGORY', 'BRANDS', 'CATEGORY','Item Code', 'Item Name','Qty','Item Rate','Total Amt']]

In [7]:
df_new.head()

,Bill Date,Cust cid,Outlet Name,MAINCATEGORY,SUBCATEGORY,BRANDS,CATEGORY,Item Code,Item Name,Qty,Item Rate,Total Amt
1,2025-03-02,1N60006307,FRESH2DAY-PALAVAKKAM,Processed Food,Jam / Spreads,NaN,Ready Foods,38784.0,F2D LEMON PICKLE 300 GM,1.00,120.0,99.00
2,2025-03-02,1N60006307,FRESH2DAY-PALAVAKKAM,Processed Food,Jam / Spreads,NaN,Ready Foods,38793.0,F2D TOMATO PICKLE 300 GM,1.00,120.0,99.00
3,2025-03-14,1N60006307,FRESH2DAY - KILPAUK,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,156.0,BANANA KARPOORAVALI,0.60,80.0,48.24
4,2025-03-14,1N60006307,FRESH2DAY - KILPAUK,FRUITS & VEGETABLES,Regular Fruits,F2D,FRESH FRUITS,156.0,BANANA KARPOORAVALI,0.09,80.0,6.96
5,2025-03-14,1N60006307,FRESH2DAY - KILPAUK,FRUITS & VEGETABLES,Herbs & Seasoning,F2D,FRESH VEGETABLES,88.0,GARLICK,0.66,169.0,112.22


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478443 entries, 1 to 478443
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Bill Date     478443 non-null  object 
 1   Cust cid      460387 non-null  object 
 2   Outlet Name   478443 non-null  object 
 3   MAINCATEGORY  478443 non-null  object 
 4   SUBCATEGORY   478443 non-null  object 
 5   BRANDS        449367 non-null  object 
 6   CATEGORY      478438 non-null  object 
 7   Item Code     478443 non-null  float64
 8   Item Name     478443 non-null  object 
 9   Qty           478443 non-null  object 
 10  Item Rate     478443 non-null  object 
 11  Total Amt     478443 non-null  object 
dtypes: float64(1), object(11)
memory usage: 43.8+ MB


In [9]:
df['Total Amt']=df['Total Amt'].str.replace(',','')

In [10]:
df['Item Code'] = df['Item Code'].astype(str)
df['Total Amt'] = pd.to_numeric(df['Total Amt'])

In [11]:
df.isna().sum()

Bill Date           0
Cust cid        18056
Outlet Name         0
MAINCATEGORY        0
SUBCATEGORY         0
BRANDS          29076
CATEGORY            5
Item Code           0
Item Name           0
Qty                 0
Item Rate           0
Total Amt       32768
dtype: int64

In [12]:
df = df.dropna()

In [13]:
df.isna().sum()

Bill Date       0
Cust cid        0
Outlet Name     0
MAINCATEGORY    0
SUBCATEGORY     0
BRANDS          0
CATEGORY        0
Item Code       0
Item Name       0
Qty             0
Item Rate       0
Total Amt       0
dtype: int64

In [14]:
df['Bill Date'] = pd.to_datetime(df['Bill Date'])

# Descriptive Statistics

In [15]:
# Step 1: First, group by CATEGORY and ITEM (or DATE) and sum sales
subgroup_totals = df.groupby(['CATEGORY'])['Total Amt'].sum().reset_index()

# Step 2: Now group by CATEGORY and get mean, min, max on the totals
category_stats = subgroup_totals.groupby('CATEGORY')['Total Amt'].agg(Mean ='mean')
category_stats =category_stats.reset_index()
category_stats = category_stats.sort_values(by = 'Mean',ascending=False).head(10)
print(category_stats)

                      CATEGORY         Mean
11                FRESH FRUITS  12584420.23
13            FRESH VEGETABLES   5746508.46
10  Exotic Fruits & Vegetables   1013514.53
3                Chilled Dairy    845492.51
16                Fresh Fruits    617712.17
37                 Ready Foods    553476.09
15                   Foodgrain    553100.59
31                     Instant    245975.10
2               Bakery & Cakes    227782.17
5               Cold Beverages    220778.85



# RFM Analysis

In [16]:
snapshot_date = df['Bill Date'].max() + pd.Timedelta(days=1)

In [17]:
rfm = df.groupby('Cust cid').agg({
    'Bill Date': lambda x: (snapshot_date - x.max()).days,
    'Item Code': 'count',
    'Total Amt': 'sum'
})

In [18]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# RFM Scoring (1–5 scale)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

# Segmenting Customers 

In [19]:
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

In [20]:
def segment_customer(score):
    if score >= 10:
        return 'High Value'
    elif score >= 5:
        return 'Medium Value'
    else:
        return 'Low Value'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

In [43]:
rfm.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score,Segment
Cust cid,,,,,,,,,
11N10000,16,21,505.63,2,5,4,254,11,High Value
11N1001,24,1,60.00,1,1,1,111,3,Low Value
11N10012,31,2,215.33,1,1,2,112,4,Low Value
11N1002,5,5,121.80,4,3,2,432,9,Medium Value
11N10030,10,3,602.07,3,2,4,324,9,Medium Value


In [21]:
print(rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}))

                Recency  Frequency     Monetary       
                   mean       mean         mean  count
Segment                                               
High Value     7.636946  26.474508  1614.751809  12965
Low Value     23.955904   1.426385    74.078127   2744
Medium Value  14.960649   3.896573   261.112637  14358


# Market Basket Analysis

In [22]:
# Prepare data: list of transactions 
transactions = df.groupby('Cust cid')['CATEGORY'].apply(list).tolist()

In [23]:
te = TransactionEncoder()
te_data = te.fit_transform(transactions)
df_te = pd.DataFrame(te_data, columns=te.columns_)

In [24]:
# Apriori and Rules
itemsets = apriori(df_te, min_support=0.01 , use_colnames=True)
rules = association_rules(itemsets, metric='confidence', min_threshold=0.5)

# Sort by lift or confidence
top_rules = rules.sort_values(by='confidence', ascending=False).head(10)


In [25]:
top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


,antecedents,consequents,support,confidence,lift
220,"(Fresh Vegetables, Masala)",(FRESH VEGETABLES),0.014401,1.000000,1.686694
417,"(Fresh Vegetables, Masala, FRESH FRUITS)",(FRESH VEGETABLES),0.013570,1.000000,1.686694
381,"(Exotic Fruits & Vegetables, Fresh Vegetables,...",(FRESH VEGETABLES),0.011175,0.997033,1.681689
499,"(Exotic Fruits & Vegetables, Fresh Vegetables,...",(FRESH VEGETABLES),0.010676,0.996894,1.681455
372,"(Foodgrain, Exotic Fruits & Vegetables, Fresh ...",(FRESH VEGETABLES),0.010244,0.996764,1.681235
322,"(General Merchandise, Chilled Dairy, Fresh Veg...",(FRESH VEGETABLES),0.010211,0.996753,1.681217
387,"(Foodgrain, Fresh Vegetables, FRESH FRUITS)",(FRESH VEGETABLES),0.018326,0.996383,1.680593
286,"(Exotic Fruits & Vegetables, Fresh Vegetables,...",(FRESH VEGETABLES),0.016796,0.996055,1.680040
250,"(FRESH FRUITS, Bakery & Cakes, Fresh Vegetables)",(FRESH VEGETABLES),0.016131,0.995893,1.679767
467,"(Exotic Fruits & Vegetables, Fresh Vegetables,...",(FRESH VEGETABLES),0.015898,0.995833,1.679666


In [26]:
# Filter itemsets of length 2
pair_itemsets = itemsets[itemsets['itemsets'].apply(lambda x: len(x) == 2)]

# Convert frozensets to strings for display
pair_itemsets['Itemset'] = pair_itemsets['itemsets'].apply(lambda x: ', '.join(sorted(x)))

# Create new DataFrame
itemsets_df = pair_itemsets[['Itemset', 'support']].rename(columns={'support': 'Support'})


/var/folders/c_/8r42hlvx2w36c1j39xf03hkw0000gp/T/ipykernel_20874/1319638728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_itemsets['Itemset'] = pair_itemsets['itemsets'].apply(lambda x: ', '.join(sorted(x)))


In [27]:
itemsets_df.head()

,Itemset,Support
25,"Bakery & Cakes, Chilled Dairy",0.034157
26,"Bakery & Cakes, Exotic Fruits & Vegetables",0.029002
27,"Bakery & Cakes, FRESH FRUITS",0.076196
28,"Bakery & Cakes, FRESH VEGETABLES",0.070775
29,"Bakery & Cakes, Foodgrain",0.017727


In [28]:
# Filter rules with 1 item in antecedents and 1 item in consequents
pairwise_rules = rules[
    (rules['antecedents'].apply(lambda x: len(x) == 1)) &
    (rules['consequents'].apply(lambda x: len(x) == 1))
].copy()

# Convert frozensets to strings
pairwise_rules['LHS'] = pairwise_rules['antecedents'].apply(lambda x: ', '.join(x))
pairwise_rules['RHS'] = pairwise_rules['consequents'].apply(lambda x: ', '.join(x))

# Select and rename columns
Rules_df = pairwise_rules[['LHS', 'RHS', 'support', 'confidence', 'lift']].rename(
    columns={'support': 'Support', 'confidence': 'Confidence', 'lift': 'Lift'}
)

In [29]:
Rules_df.sort_values(by='Lift',ascending=False).head()

,LHS,RHS,Support,Confidence,Lift
36,Fresh Vegetables,FRESH VEGETABLES,0.102272,0.965463,1.628440
42,Masala,FRESH VEGETABLES,0.051152,0.868927,1.465613
15,Eggs,FRESH VEGETABLES,0.023614,0.862697,1.455106
43,Oil & Ghee,FRESH VEGETABLES,0.018326,0.852941,1.438650
34,Foodgrain,FRESH VEGETABLES,0.069611,0.849432,1.432731


# Anova

In [30]:
# Grouping the data
high = rfm[rfm['Segment'] == 'High Value']['Monetary']
medium = rfm[rfm['Segment'] == 'Medium Value']['Monetary']
low = rfm[rfm['Segment'] == 'Low Value']['Monetary']

# One-way ANOVA
f_stat, p_value = f_oneway(high, medium, low)

print("F-statistic:", f_stat)
print("p-value:", p_value)

F-statistic: 35.530649271366194
p-value: 3.86761285877884e-16


### Market basket for High value customers

In [31]:
high_value_customers = rfm[rfm['RFM_Score'] >= 10].index

In [32]:
high_value_df = df[df['Cust cid'].isin(high_value_customers)]

In [33]:
transactions_1 = high_value_df.groupby('Cust cid')['CATEGORY'].apply(list).tolist()

In [34]:
# Use TransactionEncoder
te_data_1 = te.fit_transform(transactions_1)
df_te_1 = pd.DataFrame(te_data_1, columns=te.columns_)

# Apriori and Rules
frequent_itemsets = apriori(df_te_1, min_support=0.01, use_colnames=True)
rules_1 = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)

# Sort by lift or confidence
top_rules_1 = rules_1.sort_values(by='confidence', ascending=False).head()

In [35]:
print(top_rules_1[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                                            antecedents         consequents  \
1182  (Exotic Fruits & Vegetables, Fresh Vegetables,...  (FRESH VEGETABLES)   
2074  (Eggs, Exotic Fruits & Vegetables, Fresh Veget...  (FRESH VEGETABLES)   
1672  (Bakery & Cakes, FRESH FRUITS, General Merchan...  (FRESH VEGETABLES)   
2507  (Exotic Fruits & Vegetables, Fresh Vegetables,...  (FRESH VEGETABLES)   
1885  (Exotic Fruits & Vegetables, Fresh Vegetables,...  (FRESH VEGETABLES)   

       support  confidence      lift  
1182  0.017123         1.0  1.211003  
2074  0.010258         1.0  1.211003  
1672  0.012032         1.0  1.211003  
2507  0.013806         1.0  1.211003  
1885  0.014192         1.0  1.211003  


In [36]:
# Filter itemsets of length 2
pair_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) == 2)]

# Convert frozensets to strings for display
pair_itemsets['Itemset'] = pair_itemsets['itemsets'].apply(lambda x: ', '.join(sorted(x)))

# Create new DataFrame
itemsets_df_1 = pair_itemsets[['Itemset', 'support']].rename(columns={'support': 'Support'})


/var/folders/c_/8r42hlvx2w36c1j39xf03hkw0000gp/T/ipykernel_20874/1947400261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_itemsets['Itemset'] = pair_itemsets['itemsets'].apply(lambda x: ', '.join(sorted(x)))


In [37]:
itemsets_df_1.shape

(196, 2)

In [38]:
# Filter rules with 1 item in antecedents and 1 item in consequents
pairwise_rules = rules_1[
    (rules_1['antecedents'].apply(lambda x: len(x) == 1)) &
    (rules_1['consequents'].apply(lambda x: len(x) == 1))
].copy()

# Convert frozensets to strings
pairwise_rules['LHS'] = pairwise_rules['antecedents'].apply(lambda x: ', '.join(x))
pairwise_rules['RHS'] = pairwise_rules['consequents'].apply(lambda x: ', '.join(x))

# Select and rename columns
Rules_df_1 = pairwise_rules[['LHS', 'RHS', 'support', 'confidence', 'lift']].rename(
    columns={'support': 'Support', 'confidence': 'Confidence', 'lift': 'Lift'}
)

In [39]:
Rules_df_1.head()

,LHS,RHS,Support,Confidence,Lift
0,Bakery & Cakes,FRESH FRUITS,0.138758,0.919264,1.012596
1,Bakery & Cakes,FRESH VEGETABLES,0.133282,0.882984,1.069297
2,Chilled Dairy,FRESH FRUITS,0.231084,0.901866,0.993432
3,Chilled Dairy,FRESH VEGETABLES,0.233860,0.912703,1.105286
4,Cleaning Aids,FRESH FRUITS,0.032858,0.874743,0.963555


# Recommendation for the customer

In [40]:
def get_recommendations(user_products, rules_df):
    recommended = set()
    for _, row in rules_1.iterrows():
        antecedent = set(row['antecedents']) 
        consequent = set(row['consequents'])
        if antecedent.issubset(user_products):
            recommended.update(consequent)
    # Remove any product the user already bought
    recommended = recommended - user_products
    return list(recommended)

In [42]:
user_products = set(df[df['Cust cid'] == '11N10000']['CATEGORY'])  
recommendations = get_recommendations(user_products, rules)
print("Recommended products:", recommendations)

Recommended products: ['FRESH FRUITS']


In [41]:
high_value_customers[:5]

Index(['11N10000', '11N10036', '11N1005', '11N10050', '11N10057'], dtype='object', name='Cust cid')